# Машинное обучение, ФКН ВШЭ

## Практическое домашнее задание 3. Генерация признаков

### Общая информация

Дата выдачи: 19.10.2025

Мягкий дедлайн: 04.11.2025 23:59MSK

Жесткий дедлайн: 08.11.2025 23:59MSK

### О задании

В данном задании необходимо реализовать самые ходовые техники для создания новых признаков в табличном машинном обучении. Вам понадобиться применить всю вашу фантазию, чтобы собрать наилучшие признаки для решения задачи линейной классификации и победы в соревновании

### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 11 баллов. Детальнее про оценивание — в самом конце ноутбука.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо считываемые диаграммы.

Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### Формат сдачи
Задания сдаются через систему Anytask. Инвайт можно найти на странице курса. Присылать необходимо ноутбук с выполненным заданием. Сам ноутбук называйте в формате **homework-practice-03-features-Username.ipynb**, где Username — ваша фамилия.

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка**: ...

### **Задание 1. Вводная часть (1 балл)** <img align="center" height=25 src="https://media.tenor.com/5vGX5VO-IxsAAAAi/arthas.gif">

<img style="float: right; padding-right:10px" src="https://media.discordapp.net/attachments/674191702906503199/1429321721495949392/pomoika2.png?ex=68f5b715&is=68f46595&hm=f6664e6068c19ab5606c40ca6ef92c093cb343b8b96c471c3d8927dc4569552c&=&format=webp&quality=lossless&width=432&height=648" height=400></img>
    
В этом домашнем задании мы будем работать с датасетом по широко известной в неузких кругах видеоигре Dota 2. Если вы не знакомы с ней - ничего страшного, все необходимое для задания мы подробно опишем. Также, как ни странно, не стесняйтесь спрашивать у гугла, гпт или, прости Господи, дотеров, если вам что-то не ясно.

Подробное описание данных настолько большое, что описано отдельно на страничке соревнования.

Очень краткое описание игры:
- Dota 2 - это командная игра, в которой принимает участие 5 игроков со стороны Radiant и 5 игроков со стороны Dire;
- каждый из игроков выбирает своего героя, все герои в одной игре разные, у каждого героя есть ряд атрибутов, которые сильно отличают его от других;
- цель игроков - снести главную постройку вражеской стороны (в простонародье - трон);
- игроки могут зарабатывать деньги, убивать персонажей вражеской команды, чтобы становиться сильнее и легче снести этот самый трон;
- игра всегда заканчивается победой одной стороны, может идти от 0 минут до бесконечности.

Мы будем пытаться предсказать исход матча - победа или поражение, исходя из разных факторов, с которыми мы познакомимлся ниже. Целевая метрика уже выбрана за нас: мы будем считать ROC-AUC, но не простой, а коэффициент Джини, чтобы избегать странных значений метрики меньше $0.5$:

$$ \text{Gini} = | 2 \cdot \text{ROCAUC} - 1 |$$

#### **Важнейшее замечание!!!**

Предполагается, что у вас уже сложилось понимание:
- как крутить и вертеть данные, чтобы все трансформы ложились четко, все шейпы датафреймов стакались, все нужные джойны джойнились;
- когда ваша модель хуже или лучше, как это проверять и как подбирать гиперпараметры (валидация - наш лучший друг, иногда даже лучше настоящих);
- на какой выборке делать `fit`, а на какой `transform`, чтобы не допустить утечки данных;
- как базово кодировать признаки, чтобы ваша модель вообще обучалась (без нормализации никуда, если используем регуляризацию, категории обязательно кодируем);
- как рисовать читаемые графики.

Пожалуйста, следите за этим очень-очень внимательно, иначе рискуете получить штраф и всеобщее порицание в нашем уютном МО-1 чатике (хотя, может это и неплохо).

> Мы верим, что вы уже достаточно взрослые и самостоятельные, чтобы верно принимать жизненные решения, поэтому большая часть этой домашки не имеет четких критериев успеха, кроме факта выполненного задания.

> Максимум за эту часть домашки — 8 баллов; чтобы добить оставшиеся, вам придется посоревноваться с вашими любимыми сокурсниками.

> Впрочем, для победы в соревновании вам может понадобиться сделать все из них и, скорее всего, даже больше.

> То, насколько эти задания вам помогут, зависит исключительно от вас: какие-то преобразования признаков заходят лучше, какие-то хуже, но гарантируется, что каждое из них может помочь.

> Вы вольны делать любые пункты, которые вам больше нравятся. Для этого мы прикладываем схему, которую желательно изучить, прежде чем приступать к выполнению задания.

<img src="https://i.postimg.cc/K8qYFHyv/image-2025-10-19-164454599.png"></img>

In [ ]:
from sklearn.metrics import roc_auc_score
from typing import Optional, Callable


def gini(y_true, y_score):
    return abs(2 * roc_auc_score(y_true, y_score) - 1.0)

#### Задание 1.1. Знакомство с данными (0.5 балла)

Чтобы начать работу с данными, эти данные сперва нужно загрузить. В рамках этого ноутбука нам потребуются:

1. Информация о матчах - `matches_df_train.pq`.
2. Информация об игроках - `player_df.pq`.
3. Информация о героях - `Constants.Heroes.csv`.
4. Информация о статистиках матча - `adv_df.pq`.
5. Тестовые данные для соревнования - `matches_df_test.pq`.

Дополнительные данные, при остром желании, можно попробовать стащить [отсюда](https://www.kaggle.com/datasets/bwandowando/dota-2-pro-league-matches-2023), но в обязательную часть это не входит.

Первое задание максимально скучное:

1. Посмотрите на все датафреймы выше - какие в них колонки, по какому признаку их можно джойнить между собой, на типы и тд.
2. Проверьте, есть ли в данных пропуски, дубли, выбросы, и хотите ли вы что-то с ними сделать.
3. Разбейте данные на трейн и валидацию (доля разбиения - на ваше усмотрение).
4. Найдите, где же наш таргет, и постройте график его распределения на трейне и валидации.
5. Сделайте вывод: подходящая ли в данном случае метрика Gini? Аргументируйте ваш ответ.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 1.2. Бейзлайн (0.5 балла)

На сей раз мы будем решать задачу классификации, поэтому нам понадобится соответствующая модель. В целом, наши данные уже содержат признаки, по которым что-то даже можно построить.

1. Выберите один или несколько признаков, по которым планируете предсказывать целевую переменную (здесь не нужно придумывать ничего адового, можете даже не делать никаких трансформов, ими мы займемся позже).
2. Выберите модель по вашему вкусу, главное, чтобы она была из `sklearn.linear_model` и не была перцептроном. **Любые другие модели, кроме линейных, запрещены!!!**
3. Обучите модель и замерьте качество.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

### **Задание 2. Оптимизация для уже смешариков (2 балла)** <img align="center" src="https://cdn.7tv.app/emote/01H8RPMSBR000133946WK71YXM/1x.avif">

#### Задание 2.1. Optuna для самых маленьких (0.75 балл)

**NB: Хотя это задание и опциональное, выполнить его настоятельно рекомендуется, оно будет очень полезно сейчас, в этой дз, и вообще!**

Не стоит забывать, что у любой модели есть гиперпараметры. Конечно, львиная доля качества будет идти от фичей, но гиперпараметры списывать со счетов тоже не стоит. В конце концов, может быть, что с набором фичей, который считался безперспективным, оптимизированная модель покажет лушее качество, чем модель на топовых фичах, вот и посмотрим.

Оптимизация моделей с большим числом гиперпараметров это сложное дело. Нетрудно догадаться, что в случае, когда параметров $m$, сложность перебора в самом худшем случае будет порядка $n^m$. Подумаем, как это делать умнее.

Для эффективного подбора гиперпараметров существует несколько решений, основанных на байесовской оптимизации. В этом задании предлагается познакомиться с библиотекой [optuna](https://optuna.org/), которая делает перебор гиперпараметров легким и приятным, а также сохраняет всю историю экспериментов в одно место за вас.

Будем разбираться с этим поэтапно.

1. Напишите функцию оптимизации для логистической регрессии. Можете воспользоваться шаблоном ниже, хотя, конечно, это необязательно.
2. Подберите хотя бы 2 гиперпараметра для выбранной вами модели.

In [ ]:
def objective(trial):

    # гипепараметры модели
    model_params = {
        # ヾ(⌐■_■)ノ♪ your code here
    }

    # параметры энкодеров, если вы ими пользуетесь
    # encoder_params = {}

    # любые другие параметры, которые вы можете захотеть ввести
    # (e.g. параметры отбора фичей и прочее)
    # custom_params = {}

    return gini_valid

Убедитесь, что ваша функция работает и модель как минимум работает не хуже, чем без всякого перебора параметров. Качество может вырасти, но очень незначительно, поскольку мы все еще работаем с бейзлайном.

#### Задание 2.2. Логирование в optuna (0.75 балла)

Оптуна - это великая библиотека еще и потому, что сразу предлагает удобный инструментарий для логирования экспериментов.

Посмотрите, как в `optuna` устроена запись в хранилища, и добавьте это в пайплайн оптимизации. Без указания БД все запуски оптуны сотрутся, как только закончится сессия. Более того, так можно распараллелить процесс оптимизации на несколько машин.

Что от вас требуется:
- отдельный `storage`, который сохраняет результаты локально, а не только в рамках одной сессии. В любом удобном для вас виде: хоть датафрейм, хоть БД (в `optuna` для этого есть специальный класс, поищите).
- логирование разных экспериментов с разным именем, чтобы иметь возможность их сравнивать (например, `linreg_base` для бейзлайна регрессии, `logreg_yoba_features_ultrafinal` для самого лучшего логрега, главное, чтобы вам это было понятно и можно было адекватно сравнить перформанс разных моделей, ничего не потеряв).

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 2.3. Базовый feature selection (0.5 балла)

У любой модели, помимо этапа тюнинга во время обучения, зачастую есть и тюнинг после обучения. Готовы поспорить, что из сотен признаков, что вы потенциально можете нагенерить в данном задании, большинство окажутся вовсе бесполезными. Подробнее с методами feature selection мы познакомимся позже, но уже сейчас мы можем воспользоваться знанием, что к примеру L1-регуляризация может занулять ненужные веса, и что коэффициент у нормированных весов отражает, насколько этот вес важен.

Попробуйте встроить примитивный алгоритм feature selection:
- после обучения модели будем проверять по фиксированной сетке [25%, 50%, 75%, 100%], сколько фичей хотим оставить;
- для каждого значения сетки замеряем качество алгоритма и логгируем в оптуну значение метрики для данного процента фичей;
- для выбора оптимальной модели храним не самое лучшее значение Джини, а самое лучшее значение с учетом оставшегося количества признаков.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

Далее вы можете либо тестить фичи на дефолтной модели, либо тюнить ее после каждого изменения. Выбор, как обычно, за вами.

### **Задание 3. Простейший EDA (2.5 балла)** <img height=40px src="https://cdn3.emoji.gg/emojis/136857-pepesmoke.gif">

Чтобы построить реально балдежную модель, зачастую не получится просто жахнуть фит предиктов и пойти пить смузи, пока параметры крутятся сами. Неет, зачастую для этого требуется долгая и утомительная возня с графиками. А если необходима еще и отчетность, то и без графиков совсем не обойтись. Так и у нас. Конечно, никаких экстремальных признаков наш датасет не содержит, но и того, что есть, хватит, чтобы продемонстрировать темную сторону любого Data Science.

Задания ниже призваны скорее познакомить вас с данными, а заодно показать несколько простых способов анализировать данные, которые, в целом, никогда не помешают при работе с любыми признаками, хотя обычно так подробно смотрят лишь на самые потенциально важные факторы.

В заданиях пункта 3 всегда требуется:
- аргументировать, почему вы делаете такое решение;
- обучить модель по итогам выполнения пункта;
- сделать вывод, зашел признак или нет, и ваше мнение, почему так получилось.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 3.1. Пропуски в данных (0.5 балла)

В реальной жизни все обычно гораздо хуже, чем в вылизанных датасетах, и наши данные, увы не исключение. Колонка `duration`, которую вы могли использовать, а могли и не использовать для обучения, доступна лишь на обучающих данных, но не на тестовых. Если вы хотите опираться на длительность матча для предсказания победы, сперва эту длительность необходимо как-то достать.

У вас есть несколько возможностей это сделать:

1. Заполнить пропуски на валидационной и тестовой выборках константным значением.
2. Попробовать предсказать значение `duration` при помощи простенькой линейной модели.
3. Добавить отсутствие значения отдельным признаком.

Задание заключается в следующем:

1. Посмотрите на распределение `duration`, подберите визуализацию, которая покажет это наиболее наглядно.
2. Сделайте вывод о том, какое из выше представленных решений лучше всего поможет.
3. Заполните пропуски.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 3.2. Простейшие преобразования (0.5 балла)

В жизни рядового датасаентиста часто возникает такая ситуация, что фича, которая ну должна быть клевой, на практике оказывается тем еще отстоем. Причин тому может быть несколько: признак действительно может быть нерелевантным, в данных банально может быть ошибка; либо же мы просто неправильно этот признак готовим. Народная мудрсть (и душнилы-математики) гласит, что признаки желательно подбирать из нормального распределения в случае L2-регуляризации, тогда как на практике это достигается ой как не всегда.

Сделаем следующее:

1. Посмотрите на гистограмму признака `duration`, если вы не сделали этого до. Что вы можете сказать про смещение, среднее, хвосты распределения этого признака?
2. Изучите три распространенных способа сделать признак $y$ ($y$ здесь и ниже это `duration`, не таргет) чуть более нормальным:

$$y \rightarrow \log y,  \sqrt{y}, \frac{1}{y}$$

Посмотрите, что происходит с распределением, если применить каждое из них.

3. Попробуйте добавить все три вышеперечисленных трансформации, вместе с признаком $\exp{\log{y}}$ (это снова `duration`).

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 3.3. Регионы (0.5 балла)

Не секрет, что в отдельных регионах уровень скилла игроков отличается в лучшую или худшую сторону. Даже простые, казалось бы, признаки, иногда оказываются полезными, если распределения таргета на них по какой-то причине отличаются друг от друга.

Ваша задача:

1. Построить распределение категорий признака `region` на графике.
2. Проверить, отличается ли распределение целевой переменной для разных категорий `region`.
3. Закодируйте признак (как именно - на ваше усмотрение, но вам может очень понравиться вот [эта](https://contrib.scikit-learn.org/category_encoders) библиотека).

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 3.4. Даты (0.5 балла)

Бывалые игроки в доту знают, что игра постоянно (хотя это спорно) обновляется - в ней появляются новые механики, меняются старые и так далее. Разумно предположить, что со временем модель может вести себя чуточку по-другому. С временными изменениями часто помогает отслеживать изменение временных же признаков

Требуется:

1. Посмотреть, какие временные признаки можно достать, зная дату проведения матча
2. Подумать, есть ли какие-то знаменательные даты, которые могли повлиять на результаты матча (скажем, в одни дни игроки чуть более поддатые, или наоборот, в общем подумайте!)
2. Подумать, как именно вы хотите их кодировать (как числовые, категориальные, етц)
3. Вытащите хотя бы 2 разных временных признака из колонки `date`, добавьте хотя бы один флаг (признак со значением 1 или 0) для знаменательной даты

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 3.5. Бинаризация (0.5 балла)

В нашем мире бывает так, что один отдельный признак оказывается настолько важен, или настолько сложен, что одним весом в линейной регрессии его ну никак не описать. Тогда на помощь часто приходит бинаризация. Ее суть заключается в том, что все распределение признака бьется на непересекающиеся бины и каждый из них уже можно закодировать, например, отдельной категорией бина. Попробуем?

Сделайте следующее:

1. Возьмите признак, который, как вам кажется, может быть хорошей целью для бинаризации. Это может быть что угодно до тех пор, пока признак вещественный.
2. Бинаризуйте признак произвольным способом (равномерная сетка, квантильная сетка, ...).
3. Проверьте, зашло там или нет.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

### **Задание 4. Эмбеддинги** (2 балла) <img align="center" src="https://static.wikia.nocookie.net/dota2_gamepedia/images/1/17/Emoticon_sick.gif/revision/latest?cb=20180504011850">

Пока что мы никак не использовали информацию про героев, а ведь от них напрямую зависит исход матча, их больше 100 штук и все они разные: кто-то сильнее, кто-то слабее. Делать мы это будем при помощи эмбеддингов. Это не самая универсальная техника, но зачастую она оказывается крайне полезной, ведь в векторном пространстве определено много веселых операций, например, поиск соседей, расстояние между векторами и прочее. Здесь мы познакомимся с самым простым способом их получения через прямое кодирование нужных категорий.

#### Задание 4.1. Эмбеддер героев (1 балл)

Посмотрите на табличку `picks_bans`, в ней содержатся все герои, участвовавшие в матче (`is_pick == True`). Ваша задача - закодировать каждый матч вектором, вида:

| match_id | match_embedding
| --- | --- |
| 228 | 1 0 0 -1 -1 0 0 |

Здесь `match_embedding` - закодированный вектор матча, каждый элемент которого принимает значение 1, если герой был в команде сил света, и -1, если в команде сил тьмы, подобно `OneHotEncoder`, но не совсем. Решение вставлять его одной колонкой или разбивать вектор на n колонок оставляем за вами.

Что хочется видеть:
1. Трансформер оформлен в виде отдельной функции или класса.
2. Трансформер принимает на вход аргумент `n_heroes`, который отвечает за то, подсчитываем мы наличие одного героя или может быть сразу нескольких, кажется ничего не мешает сразу масштабировать наше преобразование.
3. Трансформер корректно работает и корректно определяет, какой игрок из какой команды.
4. Трансформер корректно применяется к трейну и валидации.

Детали реализации трансформера оставляем вам - можете воспользоваться [TransformerMixin](https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html) и подогнать его под склерновский шаблон, можете воспользоваться готовыми трансформерами, а можете писать с нуля.

In [ ]:
class HeroesEmbedder:

    def __init__(self, n_heroes: int = 1):
        self.n_heroes = n_heroes

    def fit(self, X, y=None):
        pass

    def transform(self, X, y=None):
        # ヾ(⌐■_■)ノ♪ your code here
        raise Exception("transform method not implemented")

Как проверить корректность эмбеддера:

1) Для каждого матча горизонтальная сумма должна быть 0, то есть ровно 5 героев сил Света и 5 героев сил Тьмы. Если у вас получилось не так - не спешите ставить крест на карьере дата сайентиста, а посмотрите на эти матчи подробнее, выведите их, проанализируйте и решите, что с этим делать.
2) Для каждого матча значений '1' и '-1' должно быть ровно 5, все игроки должны иметь уникальный `id`. Опять же, если это не так - покажите, где это происходит и решите, что с этим делать.

In [ ]:
embedder = HeroesEmbedder(n_heroes=1)
embeds = embedder.fit_transform(player_df)

Если вы уверены в своей правоте - присоедините эти данные к обучающей и валидационной выборке, и обучите модель.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 4.2. Эмбеддер винрейта (1 балл)

Помимо данных о присутствии героев, явно не помешает статистика, как часто эти герои выигрывают. Логично предположить, что если герой, например всеми любимый Pudge, побеждает в 99% игр, шансы на победу в будущих матчах тоже крайне высоки.

**Вопрос:** Как вы считаете, полезен ли будет этот признак, если мы уже сделали эмбеддинг героев из предыдущего пункта?

**Ответ:**

Еще одна народная мудрость, которая часто встречается в мире машинного обучения: чем лучше получается декомпозировать тот или иной признак, тем лучше. Фактически шанс на победу определенного героя, или пары-тройки-четверки героев, определяется не только тем, как часто они побеждают, но их как часто их берут, подумайте почему (необязательно вслух).

Давайте сделаем такой трансформер:

| match_id | wincount_embedding | pickcount_embedding |
| --- | --- | --- |
| 228 | 1 0 0 0 1 0 0 | 1 0 0 0 1 0 0 |
| 322 | 0.5 0 0 0 1 0 0 | 2 0 0 0 2 0 0 |
| 1337| 0.33 0 0 1 0.5 0 0 | 3 0 0 1 3 0 0 |
| ... | ... | ... |

1. Трансформер оформлен в виде класса или функции.
2. Трансформер подсчитывает **кумулятивное** число побед (`wincount_HERO`) и кумулятивное число матчей (`pickcount_HERO`) для каждого героя на тренировочной выборке и средний винрейт, средний пикрейт по герою на валидационной выборке (средние будем собирать в методе `fit`).
3. Если героя в матче нет, его винрейт не интересен, поэтому элементы эмбеддинга там будут 0.
4. Трансформер все еще может работать для связок героев, поэтому параметр `n_heroes` оставялем нетронутым.

In [ ]:
class WinrateEmbedder:

    def __init__(self, n_heroes: int = 1):
        self.n_heroes = n_heroes

    def fit(self, X, y=None):
        # ヾ(⌐■_■)ノ♪ your code here
        raise Exception("fit method not implemented")

    def transform(self, X, y=None):
        # ヾ(⌐■_■)ノ♪ your code here
        raise Exception("transform method not implemented")

Как проверить корректность эмбеддера:

1) `pickcount` и `wincount` для каждого героя должны быть неубывающими, если ваш датафрейм отсортирован по дате.
2) Для каждого матча ненулевых значений в обоих эмбеддингах должно быть не более 10.

In [ ]:
wr_embedder = WinrateEmbedder(n_heroes=1)
wr_embeds = wr_embedder.fit_transform(player_df)

Если вы считаете, что у вас все отлично, то вам также предлагается подумать - хотим ли мы делать вектор `winrate_embedding` вместо двух отдельных? Каково бы ни было ваше решение, обучите модель и замерьте качество.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

### **Задание 5. Агрегации** (2 балла) <img align="center" src="https://static.wikia.nocookie.net/dota2_gamepedia/images/4/4a/Techies_emoticon.gif/revision/latest?cb=20180504014918">

Одна из самых эффективных и в то же время громоздких трансформаций, которые можно сделать с данными - это разного рода агрегации. В первом дз мы немножко баловались с `group_by`, теперь сделаем то же самое, но уже на стероидах. Идея здесь максимально проста - схлопнуть массив данных до одного или нескольких чисел, которые эти данные описывают. Как правило, это довольно эффективная техника, но крайне громоздкая, например, для $n$ признаков, $m$ агрегаций и $d$ размера окна агрегаций, признаков получится чуть ли не $n \cdot m \cdot d$, что немало.

#### Задание 5.1. Статистики матча (0.5 балла)

Из данных, что мы все еще не использовали, есть статистики матча. На протяжении игры каждая команда зарабатывает золото и опыт. Как правило, считается, что чем циферки больше, тем больше у команды шансов на выигрыш. Разумеется, это не гарантия, но это мы сейчас и проверим. Ниже вы можете увидеть пример графика преимущества, на который мы и будем смотреть.

<img src="https://media.discordapp.net/attachments/674191702906503199/1429333425436885003/52733B32-B6C2-49FF-AABB-C3A4B3035F16.png?ex=68f5c1fc&is=68f4707c&hm=a01032723e1f1653b34f47e340ac7751d07f485b4592d0f968fc046f148a9b53&=&format=webp&quality=lossless&width=771&height=544">

Положительное значение - преимущество в пользу команды Radiant, негативное - в пользу команды Dire. Каждое значение колонки `..._adv` содержит в себе массив значений, каждый из которых представляет из себя график, или распределение, у которого есть какие-то параметры, надо бы их достать.

Сделаем вот что:
1. Посмотрите на колонки с суффиксом `_adv`.
2. Проверьте их на наличие нанов и всяких смешных краевых случаев. В первую очередь нас интересует, чтобы они не были пустыми и чтобы в них было ровно 16 значений.
3. Посчитайте простые статистики распределения, хотя бы штуки 4. Часть из них вы можете знать из курса статистики, часть вы можете найти в вашей любимой библиотеке для обработки данных, например [тут](https://docs.pola.rs/api/python/stable/reference/expressions/computation.html), какие-то признаки вы можете достать, если вспомните матан и подумаете, как фанфакты про график могут помочь модели. Для референса можете поиграть в "что хотел сказать автор" на графике выше, там есть несколько подсказок.
4. Проанализируйте, насколько это может оказаться полезным, - постройте heatmap корреляции посчитанных вами признаков с целевой переменной и между собой и сделайте вывод.

#### Задание 5.2. Тренд (0.75 балла)

Каждый уважающий себя лудоман знает, что 99% процентов игроков останавливается ровно перед тем, как сорвать джекпот. Так и здесь - если команда с треском проигрывает в первые 15 минут матча, возможно это признак камбека в следующие 50, как знать? Попробуем собрать агрегацию похитрее - она будет обозначать тренд, который есть в графиках преимущества, и если пословица верна, наша модель уловит эту зависимость.

<span style="color:grey"><font size="1">Администрация курса МО-1 категорически осуждает азартные игры, пример приводится сугубо в образовательных целях.</font></span>

Алгоритм действий:
1. Создать трансформер, который принимает на вход функцию агрегации.
2. Выделить тренд для обоих массивов `_adv` при помощи разных функций агрегаций. Попробуйте хотя бы два разных метода: разность первого и последнего значений, линейная регрессия, обученная методом МНК, нелинейная регрессия, что-то еще...
3. Добавьте хотя бы два соответствующих признака вашим трансформером (либо два разных метода, либо тренд для Radiant и тренд для Dire).
4. Проверьте, есть ли корреляция между признаком и таргетом.

In [ ]:
class TrendExtractor:

    def __init__(self, agg_func: Callable = None):
        self.agg_func = agg_func

    def fit(self, X, y=None):
        pass

    def transform(self, X, y=None):
        # ヾ(⌐■_■)ノ♪ your code here
        raise Exception("transform method not implemented")

#### Задание 5.3. Площадь под кривой и взаимодействие признаков (0.75 балла)

Скорее всего, признаки, что вы считали в предыдущих пунктах задействовали либо информацию по оси x, либо информацию по оси y, но никак не обе сразу. Один из способов попробовать учесть и то, и другое - комбинация признаков, например, можно взять и перемножить признаки между собой, либо взять и собрать кортеж из нескольких категориальных признаков. Кажется, что в пунктах с эмбеддингами и простейшим EDA это сделать совершенно не сложно, ведь признаки уже декомпозировано, а что насчет агрегаций?

Площадь под кривой, в зависимости от того, как ее считать, обладает кучей разных прикольных свойств, например, если считать ее по методу левых прямоугольников, графики, которые растут в начале, будут иметь больший скор, чем те, что растут в конце, и так далее.

Попробуем собрать трансформер поинтереснее:

1. Создать трансформер, который принимает на вход функцию интегрирования.
2. Выделить площадь для обоих массивов `_adv` при помощи двух разных функций интегрирования. Попробуйте хотя бы два разных метода: прямоугольников, трапеций, Симпсона, и т.д.
3. Добавьте хотя бы два признака (либо два метода, либо площадь для Radiant и Dire).
4. Проанализируйте самостоятельно, что важнее - преимущество в начале или в конце матча (таким образом у нас одновременно и зависимость по x, т.е. времени, и по y). Создайте кастомную функцию интегрирования (не метод прямоугольников), которая будет давать больший вест правому или левому хвосту. Добавьте этот признак.
5. Посмотрите, дает ли этот признак вклад в качество модели.

In [ ]:
class AUCExtractor:

    def __init__(self, agg_func: Callable = None):
        self.agg_func = agg_func

    def fit(self, X, y=None):
        pass

    def transform(self, X, y=None):
        # ヾ(⌐■_■)ノ♪ your code here
        raise Exception("transform method not implemented")

### **Задание 6. High tech. Low life (1 балл)** <img height=25px align="center" src="https://media1.tenor.com/m/72ScVNgTGpYAAAAC/kaneki-tokyo-ghoul.gif"></img>

Иногда, вместо того, чтобы думать, тратить свои когнитивные ресурсы, что-то анализировать и решать, хочется скинуть свою работу на кого-то другого, что, в целом, вполне разумно. Также и мы можем пойти по пути наименьшего сопротивления и попросить придумать фичи за нас, звучит крайне соблазнительно, а вот полезно ли - совсем другой вопрос.

#### Задание 6.1. Автогенерация признаков (0.5 балла)

В сети Интернет находится множество инструментов для автоматической генерации признаков. Как правило, они не приносят такого же значимого эффекта, как ручная генерация, посколько знания домена и области в библиотеки не запихнешь, но попробовать всегда можно.

Задача:
1. Изучить туториалы из библиотеки [`featuretools`](https://featuretools.alteryx.com/en/stable).
2. Собрать 2-3 признака при помощи DFS или любого другого генератора.
3. Замерить качество и задуматься, зачем нужна эта домашка.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

#### Задание 6.2. Нейросетевые признаки (0.5 балла)

Если вы следите за новостями, то вы, наверное, заметили, что в последнее время появилась такая модная и хайповая вещь, как большие языковые модели. Злые языки утверждают, что обычному работяге фит предиктору не место в мире будущего, где господствует ИИ. Давайте в этом (раз)убедимся.

Попробуйте:
1. Спросить у вашей любимой нейросети, какие признаки она может для вас придумать. Можете опираться на пункты выше, можете придумать что-то свое. Но помните, что как говорится, какой стол, такой и стул, поэтому пишите промпты с умом.
2. Показать, что нейросеть вам посоветовала, и реализовать это
3. Проанализировать результат и сделать твердый и четкий вывод, заменят ли нас LLM-ки.

In [ ]:
# ヾ(⌐■_■)ノ♪ your code here

### Заключение

Чтобы победить в соревновании, вам может понадобиться куда большее количество признаков. Не стесняйтесь гуглить, масштабировать пункты выше, например, эмбеддинги, пробовать применять те же техники, но к другим признакам, пробовать разные схемы валидации, тюнить в конце концов. Единого рецепта для лучшего решения задачи, увы, не существует, это горькая правда всего машинного обучения

In [ ]:
# ваши впечатления от задания

### А теперь об оценивании

Каждая из задач в ноутбуке имеет свою стоимость (указана в скобках рядом с задачей). При этом важно уточнить разницу между баллами за ноутбук и дополнительными баллами за позицию на приватном лидерборде в соревновании на Kaggle:

1. **Максимум за код/ноутбук — 8.0 баллов.**
   То есть, независимо от суммарной теоретической суммы всех подпунктов в тексте задания, за реализацию в ноутбуке можно получить не более 8 баллов.

2. **Максимально возможная оценка за всю работу — 11.0 баллов.**
   Остальные до 3.0 баллов начисляются за результаты в соревновании на Kaggle (лидерборд), при выполненном и загруженном в систему Anytask ноутбуке.

Баллы за сореву состоят из двух частей: процентильный бонус (до 2.0 баллов) и бонус за попадание в топ-10 (до 1.0 балла). Суммарный вклад соревнования не может превышать 3.0 баллов.

**A. Процентильный балл (не суммируется):**

* Если вы обогнали ≥ 90% участников (т.е. попали в топ 10%) — +2.0 балла.
* Если вы обогнали ≥ 60% участников — +1.5 балла.
* Если вы обогнали ≥ 30% участников — +1.0 балла.
* Если вы обогнали ≥ 10% участников — +0.5 балла.

**B. Балл за попадание в топ-10 (дополнительно):**

* 1-е место — +1.00 балла
* 2-е–3-е место — +0.75 балла
* 4-е–7-е место — +0.50 балла
* 8-е–10-е место — +0.25 балла

Пример расчёта

* Вы сделали ноутбук и получили за него 7.0 / 8.0.
* На лидерборде вы, зайка, обогнали 10% участников → процентильный бонус +2.0.
* Ваша позиция — 3-е место → топ-10 бонус +0.75.
* Итого: 7.0 + 2.0 + 0.75 = 9.75.

**Требование к воспроизводимости**

Баллы за соревнование начисляются **только** при наличии готового, полностью воспроизводимого пайплайна, который подтверждает результат лучшего сабмита. Такое решение нужно сдавать вместе с ноутбуком и своим ников в kaggle в Anytask ассистенту. Пайплайн должен выполнять всё автоматически при запуске ноутбука: при последовательном исполнении всех ячеек ноутбука (без ручных вмешательств) он должен воспроизвести предобработку, обучение/инференс и сгенерировать итоговый CSV-файл с прогнозами, используемый для сабмита. Среда запуска - colab.